In [17]:
from time import perf_counter
from math import log
from collections import defaultdict

In [18]:
def safe_log(x):
    return log(x) if x != 0 else float('-inf')

In [19]:
def read_train(filename):
    with open(filename) as f:
        data = f.readlines()

    n = len(data) - 1
    
    # read column headers
    line = data[0].strip().split(", ")
    attr_vars = line[:-1]
    class_var = line[-1]
    
    class_attr_counts = defaultdict( lambda: [defaultdict(int) for _ in attr_vars] )
    class_counts = defaultdict(int)
    
    attr_domains = [set() for _ in attr_vars]
    class_domain = set()

    # read data and update counts
    for line in data[1:]:
        line = line.strip().split(", ")
        c = line[-1]
        class_counts[c] += 1
        class_domain.add(c)
        for i, attr_val in enumerate(line[:-1]):
            class_attr_counts[c][i][attr_val] += 1
            attr_domains[i].add(attr_val)
    
    return n, attr_vars, class_var, class_attr_counts, class_counts, attr_domains, class_domain

In [20]:
def smoothed(smoothing, amt, val):
    if smoothing:
        return val + amt
    else:
        return val

In [21]:
def model_str(n, attr_vars, class_var, class_attr_counts, class_counts, attr_domains, class_domain, smoothing):
    res = []
    
    for c in class_domain:
        res.append(f"P({class_var}={c}) = {class_counts[c]/n}")
        for i, attr in enumerate(class_attr_counts[c]):
            total_s = smoothed(smoothing, len(attr_domains[i]), sum(attr.values()))
            for key in attr_domains[i]:
                val_s = smoothed(smoothing, 1, attr[key])
                res.append(f"P({attr_vars[i]}={key} | {class_var}={c}) = {val_s / total_s}")
    
    return '\n'.join(res)

In [22]:
def classify(file_in, file_out, n, class_attr_counts, class_counts, attr_domains, class_domain, smoothing):
    with open(file_in) as f:
        data = f.readlines()
    
    res = []
    
    for line in data[1:]:
        line = line.strip().split(", ")
        
        likelihoods = {c: safe_log(class_counts[c]/n) for c in class_domain}  # logs are used to prevent numerical underflow
        for c in class_domain:
            for i, attr_val in enumerate(line):
                likelihoods[c] += safe_log(smoothed(
                    smoothing, 
                    1, 
                    class_attr_counts[c][i][attr_val]))
                likelihoods[c] -= safe_log(smoothed(
                    smoothing, 
                    len(attr_domains[i]), 
                    sum(class_attr_counts[c][i].values())))
        
        res.append(max(likelihoods, key=lambda x: likelihoods[x]))
    
    with open(file_out, 'w') as f:
        f.write('\n'.join(res))

In [26]:
smoothing = True

In [27]:
start = perf_counter()
file_train = "naive_bayes_train.txt"
n, attr_vars, class_var, class_attr_counts, class_counts, attr_domains, class_domain = read_train(file_train)
train_res = model_str(n, attr_vars, class_var, class_attr_counts, class_counts, attr_domains, class_domain, smoothing)
end = perf_counter()

print(f"time elapsed (s): {end - start}\n")
print(train_res)

time elapsed (s): 0.0021679999999832944

P(C=banana) = 0.3125
P(A1=a | C=banana) = 0.2222222222222222
P(A1=c | C=banana) = 0.3333333333333333
P(A1=b | C=banana) = 0.3333333333333333
P(A1=d | C=banana) = 0.1111111111111111
P(A2=o | C=banana) = 0.125
P(A2=m | C=banana) = 0.125
P(A2=n | C=banana) = 0.75
P(A3=y | C=banana) = 0.2222222222222222
P(A3=z | C=banana) = 0.1111111111111111
P(A3=w | C=banana) = 0.2222222222222222
P(A3=x | C=banana) = 0.4444444444444444
P(C=cucumber) = 0.1875
P(A1=a | C=cucumber) = 0.14285714285714285
P(A1=c | C=cucumber) = 0.42857142857142855
P(A1=b | C=cucumber) = 0.2857142857142857
P(A1=d | C=cucumber) = 0.14285714285714285
P(A2=o | C=cucumber) = 0.5
P(A2=m | C=cucumber) = 0.3333333333333333
P(A2=n | C=cucumber) = 0.16666666666666666
P(A3=y | C=cucumber) = 0.14285714285714285
P(A3=z | C=cucumber) = 0.2857142857142857
P(A3=w | C=cucumber) = 0.42857142857142855
P(A3=x | C=cucumber) = 0.14285714285714285
P(C=apple) = 0.5
P(A1=a | C=apple) = 0.4166666666666667
P(A1=

In [28]:
file_test = "naive_bayes_test.txt"
file_out = "naive_bayes_output.txt"

start = perf_counter()
classify(file_test, file_out, n, class_attr_counts, class_counts, attr_domains, class_domain, smoothing)
end = perf_counter()

print(f"time elapsed (s): {end - start}\n")
print(f"written to file {file_out}")

time elapsed (s): 0.00474949999988894

written to file naive_bayes_output.txt
